# 5 exemples pratiques de décorateurs Python

## 1. Le décorateur timer
Mesurez simplement le temps d'exécution de vos fonctions.

In [1]:
import time
from functools import wraps

def timer(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        debut = time.perf_counter()
        resultat = func(*args, **kwargs)
        fin = time.perf_counter()
        duree = fin - debut
        print(f"{func.__name__} a pris {duree:.4f} secondes")
        return resultat
    return wrapper

# Exemple d'utilisation
@timer
def operation_longue(n):
    time.sleep(n)  # Simule un traitement long
    return "terminé"

# Test
resultat = operation_longue(2)

operation_longue a pris 2.0020 secondes


## 2. Le décorateur auth
Gérez l'authentification de manière élégante.

In [2]:
from functools import wraps
from typing import Dict, Any, Callable

def auth(role_requis: str):
    def decorateur(func: Callable):
        @wraps(func)
        def wrapper(utilisateur: Dict[str, Any], *args, **kwargs):
            if not utilisateur.get('role'):
                raise ValueError("Utilisateur non authentifié")
            
            if utilisateur['role'] != role_requis:
                raise ValueError(f"Rôle {role_requis} requis")
                
            return func(utilisateur, *args, **kwargs)
        return wrapper
    return decorateur

# Exemple d'utilisation
@auth(role_requis='admin')
def supprimer_utilisateur(utilisateur: Dict, user_id: int):
    return f"Utilisateur {user_id} supprimé par {utilisateur['nom']}"

# Tests
def tester_auth():
    # Préparation des utilisateurs de test
    admin = {'nom': 'Alice', 'role': 'admin'}
    user = {'nom': 'Bob', 'role': 'user'}
    anonyme = {'nom': 'Anonymous'}  # Sans rôle

    print("Test 1: Admin (devrait réussir)")
    try:
        resultat = supprimer_utilisateur(admin, 123)
        print(f"✅ Succès: {resultat}")
    except ValueError as e:
        print(f"❌ Erreur inattendue: {e}")

    print("\nTest 2: Utilisateur simple (devrait échouer)")
    try:
        supprimer_utilisateur(user, 123)
        print("❌ Erreur: L'utilisateur simple ne devrait pas avoir accès")
    except ValueError as e:
        print(f"✅ Erreur attendue: {e}")

    print("\nTest 3: Utilisateur sans rôle (devrait échouer)")
    try:
        supprimer_utilisateur(anonyme, 123)
        print("❌ Erreur: L'utilisateur anonyme ne devrait pas avoir accès")
    except ValueError as e:
        print(f"✅ Erreur attendue: {e}")

# Exécution des tests
tester_auth()

Test 1: Admin (devrait réussir)
✅ Succès: Utilisateur 123 supprimé par Alice

Test 2: Utilisateur simple (devrait échouer)
✅ Erreur attendue: Rôle admin requis

Test 3: Utilisateur sans rôle (devrait échouer)
✅ Erreur attendue: Utilisateur non authentifié


## 3. Le décorateur cache
Mémorisez les résultats des fonctions coûteuses.

In [3]:
from functools import wraps
from typing import Dict, Any, Callable

def cache(func: Callable):
    resultats: Dict[Any, Any] = {}
    
    @wraps(func)
    def wrapper(*args, **kwargs):
        # Crée une clé unique pour les arguments
        key = str(args) + str(kwargs)
        
        if key not in resultats:
            print(f"Calcul de {func.__name__}{args}")
            resultats[key] = func(*args, **kwargs)
        else:
            print(f"Utilisation du cache pour {func.__name__}{args}")
            
        return resultats[key]
    return wrapper

# Exemple d'utilisation
@cache
def fibonacci(n: int) -> int:
    if n < 2:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

# Test

print(fibonacci(10))  # Premier appel
print(fibonacci(10))  # Utilise le cache

Calcul de fibonacci(10,)
Calcul de fibonacci(9,)
Calcul de fibonacci(8,)
Calcul de fibonacci(7,)
Calcul de fibonacci(6,)
Calcul de fibonacci(5,)
Calcul de fibonacci(4,)
Calcul de fibonacci(3,)
Calcul de fibonacci(2,)
Calcul de fibonacci(1,)
Calcul de fibonacci(0,)
Utilisation du cache pour fibonacci(1,)
Utilisation du cache pour fibonacci(2,)
Utilisation du cache pour fibonacci(3,)
Utilisation du cache pour fibonacci(4,)
Utilisation du cache pour fibonacci(5,)
Utilisation du cache pour fibonacci(6,)
Utilisation du cache pour fibonacci(7,)
Utilisation du cache pour fibonacci(8,)
55
Utilisation du cache pour fibonacci(10,)
55


## 4. Le décorateur logger
Tracez facilement l'exécution de vos fonctions.

In [4]:
import logging
from functools import wraps
from typing import Callable
from datetime import datetime

logging.basicConfig(level=logging.INFO)

def logger(func: Callable):
    @wraps(func)
    def wrapper(*args, **kwargs):
        nom_fonction = func.__name__
        horodatage = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        logging.info(f"[{horodatage}] Appel de {nom_fonction}")
        logging.info(f"Arguments: {args}, {kwargs}")
        
        try:
            resultat = func(*args, **kwargs)
            logging.info(f"Résultat: {resultat}")
            return resultat
        except Exception as e:
            logging.error(f"Erreur dans {nom_fonction}: {str(e)}")
            raise
        
    return wrapper

# Exemple d'utilisation
@logger
def diviser(a: float, b: float) -> float:
    return a / b

# Tests
diviser(10, 2)
try:
    diviser(10, 0)
except ZeroDivisionError:
    pass

INFO:root:[2024-11-12 00:24:38] Appel de diviser
INFO:root:Arguments: (10, 2), {}
INFO:root:Résultat: 5.0
INFO:root:[2024-11-12 00:24:38] Appel de diviser
INFO:root:Arguments: (10, 0), {}
ERROR:root:Erreur dans diviser: division by zero


## 5. Le décorateur retry
Gérez automatiquement les tentatives en cas d'échec.

In [5]:
import time
from functools import wraps
from typing import Callable, Optional, Type, Union, Tuple
import random

def retry(
    tentatives: int = 3,
    delai: float = 1,
    exceptions: Union[Type[Exception], Tuple[Type[Exception], ...]] = Exception
):
    def decorateur(func: Callable):
        @wraps(func)
        def wrapper(*args, **kwargs):
            for i in range(tentatives):
                try:
                    return func(*args, **kwargs)
                except exceptions as e:
                    if i == tentatives - 1:  # Dernière tentative
                        print(f"Échec après {tentatives} tentatives")
                        raise
                    print(f"Tentative {i+1} échouée: {str(e)}")
                    time.sleep(delai)
            return None
        return wrapper
    return decorateur

# Exemple d'utilisation
@retry(tentatives=3, delai=1, exceptions=(ConnectionError, TimeoutError))
def appel_api_instable() -> str:
    if random.random() < 0.7:  # 70% de chance d'échec
        raise ConnectionError("Erreur de connexion")
    return "Succès !"

# Test
try:
    resultat = appel_api_instable()
    print(f"Résultat final: {resultat}")
except ConnectionError:
    print("Échec de toutes les tentatives")

Tentative 1 échouée: Erreur de connexion
Tentative 2 échouée: Erreur de connexion
Résultat final: Succès !


## Conseils d'utilisation

- Utilisez `functools.wraps` pour préserver les métadonnées de la fonction décorée
- Pensez aux types hints pour la maintenabilité
- Privilégiez des exceptions spécifiques plutôt que génériques
- Évitez les effets de bord dans vos décorateurs
- N'oubliez pas que les décorateurs sont appelés à l'import du module

## Pour aller plus loin

- Combinez plusieurs décorateurs (par exemple `@logger` et `@timer`)
- Créez des décorateurs avec des paramètres
- Utilisez des décorateurs sur des classes
- Explorez les décorateurs asynchrones avec `async/await`